In [309]:
import torch
import pandas as pd
from sklearn.metrics.pairwise import cosine_distances
from sklearn.neighbors import NearestNeighbors

In [108]:
# Загрузка эмбеддингов из файла
embeddings = torch.load('embeddings.pth')

In [109]:
embeddings[1]

tensor([-0.5929, -0.5890,  0.3074, -0.1446, -0.0060, -0.3090, -0.1016,  0.0486,
        -0.2660, -0.4252,  1.1846, -0.1723,  0.0310, -0.5860,  0.8490,  0.5920,
         0.6755, -0.1215, -0.2984,  0.2459,  0.0699, -0.1027,  0.9846,  0.6645],
       grad_fn=<SelectBackward0>)

In [110]:
# Читаем файл с компетенциями
vocab = pd.read_csv('vocabulary.csv')

In [111]:
vocab.head(3)

,id,skill
0,0,[PAD]
1,1,[MASK]
2,2,NET FRAMEWORK


In [112]:
len(embeddings)

12031

In [113]:
embeddings.shape

torch.Size([12031, 24])

In [114]:
# Читаем файл с компетенциями
df = pd.read_excel('Dataset_L_8_15_diff_8_COLAB.xlsx')

In [115]:
df['SKILLS'] = df['SKILLS'].astype(str)

# Длинна каждого посчитаем длинну
df['list_of_skill'] = df['SKILLS'].apply(lambda x: x.split(','))

In [116]:
df['cnt_skill'] = df['list_of_skill'].apply(lambda x: len(x))

In [117]:
df_skill = df[(df.cnt_skill > 7) & (df.cnt_skill < 16)]

In [118]:
df_skill.head(3)

,SKILLS,list_of_skill,cnt_skill
0,"NET FRAMEWORK,C,MS SQL SERVER,SQL,REACT,REDUX,...","[NET FRAMEWORK, C, MS SQL SERVER, SQL, REACT, ...",12
1,"ЯНДЕКС МЕТРИКА,ЯНДЕКС ДИРЕКТ,GOOGLE ANALYTICS,...","[ЯНДЕКС МЕТРИКА, ЯНДЕКС ДИРЕКТ, GOOGLE ANALYTI...",13
2,"ЯНДЕКС ДИРЕКТ,ЯНДЕКС МЕТРИКА,КОНТЕКСТНАЯ РЕКЛА...","[ЯНДЕКС ДИРЕКТ, ЯНДЕКС МЕТРИКА, КОНТЕКСТНАЯ РЕ...",13


In [301]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import torch
import numpy as np

class VectorProcessor:
    def __init__(self, vocab, embeddings, df_skill, skill_find):
        self.vocab = vocab
        self.embeddings = embeddings
        self.df_skill = df_skill
        self.skill_find = skill_find

    def get_word_vectors(self, words):
        vocab_list = self.vocab['skill'].to_list()
        word_indices = [vocab_list.index(word) if word in vocab_list else -1 for word in words]
        word_indices = [idx for idx in word_indices if idx != -1]  # Убираем невалидные индексы
        word_vectors = self.embeddings[word_indices]
        return word_vectors.clone().detach()

    def average_vector(self, result_tensor):
        avg_vector = torch.mean(result_tensor, dim=0)
        return torch.Tensor(avg_vector)

    def get_pca_vector(self, result_tensor, num_components=7):
        data = result_tensor.cpu().detach().numpy()
        pca = PCA(n_components=num_components)
        pca.fit(data)
        pca_vector = torch.Tensor(pca.components_[0])
        return pca_vector

    def get_combined_tsne_vector(self, vectors, perplexity=None):
        data = vectors.cpu().detach().numpy()
        if perplexity is None:
            perplexity = min(5, len(vectors) - 1)
        tsne = TSNE(n_components=1, perplexity=perplexity)
        tsne_result = tsne.fit_transform(data)
        tsne_vector = torch.tensor(tsne_result.ravel())
        return tsne_vector
    
    def get_k_cosine_distances(self, input_vector, embeddings, k=5):
        cosine_dists = cosine_distances([input_vector], embeddings)
        sorted_indices = np.argsort(cosine_dists)
        k_indices = sorted_indices[0][:k]
        k_distances = cosine_dists[0][k_indices]
        return k_distances, k_indices
    
    def get_k_nearest_neighbors(self, input_vector, embeddings, k=5):
        neigh = NearestNeighbors(n_neighbors=k)
        neigh.fit(embeddings)
        distances, indices = neigh.kneighbors([input_vector])
        return distances[0], indices[0]
   
    def process_data(self):
        batch_size=500
        total_rows = len(self.df_skill)
        processed_rows = 0
        
        self.df_skill['average_vector'] = None  # Добавляем пустое поле 'average_vector'
        self.df_skill['pca_vector'] = None  # Добавляем пустое поле 'pca_vector'
        self.df_skill['tsne_vector'] = None  # Добавляем пустое поле 'tsne_vector
        for index, row in self.df_skill.iterrows():
            words = row['list_of_skill']  # list_of_skill - это список навыков, для которых нужно получить векторы
            word_vectors = self.get_word_vectors(words)
            avg_vector = self.average_vector(word_vectors)
            pca_vector = self.get_pca_vector(word_vectors)
            tsne_vector = self.get_combined_tsne_vector(word_vectors)

            self.df_skill.at[index, 'average_vector'] = avg_vector
            self.df_skill.at[index, 'pca_vector'] = pca_vector
            self.df_skill.at[index, 'tsne_vector'] = tsne_vector
            
            processed_rows += 1

            if processed_rows % batch_size == 0:
                print(f'Processed {processed_rows} out of {total_rows} rows.')

        remaining_rows = total_rows - processed_rows
        print(f'Processed all {total_rows} rows. {remaining_rows} rows remaining.')
        
    def find_cv(self, skill_find):
        input_vector = get_word_vectors(skill_find)
        avg_vector = self.average_vector(word_vectors) # метод средних
        pca_vector = self.get_pca_vector(word_vectors) # метод 'pca_vector'
        tsne_vector = self.get_combined_tsne_vector(word_vectors) # метод 'tsne_vector
        
        average_vector_list = df_skill['average_vector'].to_list()
        pca_vector_list = df_skill['pca_vector'].to_list()
        tsne_vector_list = df_skill['tsne_vector'].to_list()
              
        # Преобразование списка тензоров в один тензор                                           
        tensor_column_avg = torch.stack(average_vector_list)
        tensor_column_pca = torch.stack(pca_vector_list)
        
        # Находим максимальную длину вектора в списке
        max_len = max(len(tensor) for tensor in tsne_vector_list)
        # Выравниваем тензоры до одинаковой длины, добавляя нулевые значения при необходимости
        padded_tsne_vector_list = [torch.cat((tensor, torch.zeros(max_len - len(tensor)))) if len(tensor) < max_len else tensor for tensor in tsne_vector_list]
        # Объединяем тензоры в список в один тензор
        tensor_column_tsne = torch.stack(padded_tsne_vector_list)
        
        
        # Определяем текущую длину tsne_vector
        current_len = len(tsne_vector)

        # Вычисляем, сколько нулей нужно добавить
        num_zeros_to_add = max_len - current_len

        # Создаем тензор с нулевыми значениями для дополнения
        zeros_to_add = torch.zeros(num_zeros_to_add)

        # Добавляем нули к tsne_vector
        padded_tsne_vector = torch.cat((tsne_vector, zeros_to_add))

        print(f'Поиск методом K-ближайших соседей 5-ти ближайших резюме. Метод получения резюме - average ')
        lengt, spisok = self.get_k_nearest_neighbors(avg_vector.detach().numpy(), tensor_column_avg, k=5)
        for i in range (len(spisok)):
             print(f' Резюме {i+1} ', df_skill['list_of_skill'][spisok[i]])
                
        print(f'Поиск методом K-ближайших соседей 5-ти ближайших резюме. Метод получения резюме - PCA')
        lengt, spisok = self.get_k_nearest_neighbors(pca_vector.detach().numpy(), tensor_column_pca, k=5)
        for i in range (len(spisok)):
             print(f' Резюме {i+1} ', df_skill['list_of_skill'][spisok[i]])
                
        print(f'Поиск методом K-ближайших соседей 5-ти ближайших резюме. Метод получения резюме - T-SNE')
        lengt, spisok = self.get_k_nearest_neighbors(padded_tsne_vector.detach().numpy(), tensor_column_tsne, k=5)
        for i in range (len(spisok)):
             print(f' Резюме {i+1} ', df_skill['list_of_skill'][spisok[i]])
                
        print(f'Поиск методом косинусного расстояния 5-ти ближайших резюме. Метод получения резюме - average')
        lengt, spisok = self.get_k_cosine_distances(avg_vector.detach().numpy(), tensor_column_avg, k=5)
        for i in range (len(spisok)):
             print(f' Резюме {i+1} ', df_skill['list_of_skill'][spisok[i]])
                
        print(f'Поиск методом косинусного расстояния 5-ти ближайших резюме. Метод получения резюме - PCA')
        lengt, spisok = self.get_k_cosine_distances(pca_vector.detach().numpy(), tensor_column_pca, k=5)
        for i in range (len(spisok)):
             print(f' Резюме {i+1} ', df_skill['list_of_skill'][spisok[i]])
                
        print(f'Поиск методом косинусного расстояния 5-ти ближайших резюме. Метод получения резюме - T-SNE')
        lengt, spisok = self.get_k_cosine_distances(padded_tsne_vector.detach().numpy(), tensor_column_tsne, k=5)
        for i in range (len(spisok)):
             print(f' Резюме {i+1} ', df_skill['list_of_skill'][spisok[i]])
        

In [302]:
skill_find = df_skill['list_of_skill'][23]

In [303]:
vector_processor = VectorProcessor(vocab, embeddings, df_skill, skill_find)
#vector_processor.process_data()

In [308]:
vector_processor.find_cv(skill_find)

Поиск методом K-ближайших соседей 5-ти ближайших резюме. Метод получения резюме - average 
 Резюме 1  ['ЯНДЕКС МЕТРИКА', 'ЯНДЕКС ДИРЕКТ', 'GOOGLE ANALYTICS', 'GOOGLE ADWORDS', 'SEO', 'E MAIL MARKETING', 'УПРАВЛЕНИЕ ИНТЕРНЕТ ПРОЕКТАМИ', 'АНАЛИТИЧЕСКОЕ МЫШЛЕНИЕ', 'INTERNET MARKETING', 'ПРОДВИЖЕНИЕ САЙТОВ', 'УПРАВЛЕНИЕ БЮДЖЕТОМ', 'УПРАВЛЕНИЕ КОМАНДОЙ', 'WEB АНАЛИТИКА']
 Резюме 2  ['РАЗВИТИЕ ПРОДАЖ', 'ИНТЕРНЕТ РЕКЛАМА', 'ПОИСК И ПРИВЛЕЧЕНИЕ КЛИЕНТОВ', 'КОНТЕКСТНАЯ РЕКЛАМА', 'INTERNET MARKETING', 'ИНТЕРНЕТ МАРКЕТИНГ', 'МАРКЕТИНГОВЫЙ АНАЛИЗ', 'АНАЛИТИЧЕСКОЕ МЫШЛЕНИЕ', 'CUSTOMER JOURNEY MAP', 'ЯНДЕКС ДИРЕКТ', 'B2B МАРКЕТИНГ', 'E MAIL MARKETING', 'EMAIL МАРКЕТИНГ', 'BTL']
 Резюме 3  ['ИНТЕРНЕТ РЕКЛАМА', 'ЯНДЕКС МЕТРИКА', 'GOOGLE ANALYTICS', 'ТАРГЕТИНГ', 'ВЕДЕНИЕ ОТЧЕТНОСТИ', 'INTERNET MARKETING', 'АНАЛИТИЧЕСКОЕ МЫШЛЕНИЕ', 'СОЦИАЛЬНЫЕ СЕТИ', 'ОПТИМИЗАЦИЯ', 'ПРАВИЛЬНО РАССТАВЛЯТЬ ПРИОРИТЕТЫ', 'ЖЕЛАНИЕ УЧИТЬСЯ И РАЗВИВАТЬСЯ', 'ЭФФЕКТИВНОСТЬ', 'ДОСТИЖЕНИЕ ПОСТАВЛЕННЫХ ЦЕЛЕЙ', 'ПЛАНИРОВАНИЕ И ОРГАН